In [1]:
#!pip install tensorflow==2.8
#!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install keras-cv-attention-models
!pip install tf-models-official
!pip install pylibjpeg
!pip install python-gdcm
!pip install ipywidgets
!pip install pydicom
!pip install -qU scikit-learn
# !pip install --upgrade google-cloud-storage
#!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist/cuda/10.0 nvidia-dali
#!pip install tensorflow-io
# !pip install -U jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.6/688.6 kB 11.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 30.2 MB/s eta 0:00:00
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl size=93512 sha256=00f226590afaec320efb38b8c59dcc84711ecd645bbf1b6d1469e801b2681304
  Stored in directory: /root/.cache/pip/wheels/58/e6/f9/f87c8f8be098b51

In [2]:
# import tensorflow as tf
# tf.keras.backend.clear_session()
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver("local")  # TPU detection
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# strategy = tf.distribute.TPUStrategy(tpu)

In [3]:
!nvidia-smi

Tue Jun 20 15:35:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:

import tensorflow as tf
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_cpu_global_jit'
#from keras_cv_attention_models import convnext, efficientnet, swin_transformer_v2
import pandas as pd
from pydicom import dcmread
#import Pillow
import pylibjpeg
import numpy as np
import cv2 
import gc 
import gdcm
import matplotlib.pyplot as plt
import matplotlib.image
from sklearn.model_selection import train_test_split
#from tensorflow.keras import initializers
#import official.nlp.modeling.layers as nlp_layers

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

True
True


In [5]:
import tensorflow as tf
import time
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus, 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        strategy = tf.distribute.MirroredStrategy()
        num_replicas_in_sync = strategy.num_replicas_in_sync
        print('number of replicas: ',num_replicas_in_sync)
    except RuntimeError as e:
        print(e)
else:
    num_replicas_in_sync = 1
    strategy = tf.distribute.get_strategy()

1 Physical GPUs, 1 Logical GPU
number of replicas:  1


In [6]:
class Distribute_train_vali_data():
    def __init__(self, total_data_points, train_vali_ratio, random_object):
        self.total_data_points = total_data_points
        self.train_vali_ratio = train_vali_ratio
        self.random_object = random_object
        
    def _get_pair(self,input_tensor, element_position, duplicate_constant, reduce_shape):
        """element_position = index in list"""
        _new_length = reduce_shape - (element_position+1)
        
        left_list = input_tensor[element_position+1:reduce_shape,0:2]    #left_list: the remain tensors in list
        density_list = input_tensor[element_position+1:reduce_shape,4:5]
        
        current_value = input_tensor[element_position]
        remained_elements = tf.concat([left_list, density_list],axis = 1)
        
        #Create duplcate information
        """
        Example:
                        "dup_infor" "index_data"
        duplicated_index =  [[3     ,    0]   --> row index 0 repeats 3 times
                             [2     ,    1]   --> row index 1 repeats 2 times
                             [5     ,    2]   --> row index 2 repeats 5 times
                             [4     ,    3]]  --> row index 3 repeats 4 times
        
        """
        dup_infor = tf.multiply(tf.add(current_value[4],density_list),duplicate_constant)   #from density level of current element and other elements
        index_data = tf.reshape(tf.range(_new_length), shape = (_new_length,1) )
        duplicated_index = tf.map_fn(lambda x: tf.fill(dims = [x[0]], value = x[1]), 
                                 elems = tf.concat([dup_infor, index_data],axis = 1),
                                 fn_output_signature = tf.RaggedTensorSpec(shape= [None], dtype=tf.int32) )
        """
        Example:
        duplicated_index = [0,0,0,1,1,2,2,2,2,2,4,4,4]
        """
        duplicated_index = duplicated_index.merge_dims(0,1)
        
        return tf.map_fn(lambda x: tf.concat([current_value,x],axis = 0) , elems = tf.gather(params = remained_elements, indices = duplicated_index, axis = 0))

    def _query_lateral_tensors_from_frame(self,data_frame):
        """Query element in dataframe and convert into tensor"""
        Left_list = data_frame.query( 'laterality == 0', inplace = False)
        Left_tensors = tf.convert_to_tensor(Left_list.values, dtype = tf.int32)
        Right_list = data_frame.query( 'laterality == 1', inplace = False)
        Right_tensors = tf.convert_to_tensor(Right_list.values, dtype = tf.int32)
        return Left_tensors, Right_tensors
    
    def _get_mix_tensors_for_training(self,pos_train_frame, neg_train_frame, duplicate_constant):
        """Inputs: positive class and negative class in dataframe"""
        Left_tensors , Right_tensors = self._query_lateral_tensors_from_frame(pos_train_frame)
        
        ini_length_L = Left_tensors.shape[0]
        ini_length_R = Right_tensors.shape[0]
        tf.print("Befor applying MixUP method, in positive class: ")
        tf.print("   Laterality L view: ",ini_length_L,"images")
        tf.print("   Laterality R view: ",ini_length_R,"images")
        Negative_class_tensor = tf.convert_to_tensor(neg_train_frame.values, dtype = tf.int32) 
        
        #Create pairs of elements 
        Left_Positive = tf.concat([self._get_pair(Left_tensors, element_position = i, duplicate_constant = duplicate_constant, reduce_shape = ini_length_L) for i in range(ini_length_L-3)], axis = 0)
        Right_Positive = tf.concat([self._get_pair(Right_tensors, element_position = i, duplicate_constant = duplicate_constant, reduce_shape = ini_length_R) for i in range(ini_length_R-3)], axis = 0)
        Negative_class_tensor = tf.concat([Negative_class_tensor, tf.zeros(shape = (Negative_class_tensor.shape[0],3), dtype = tf.int32) ], axis = 1)
        
        #Add zeros (no argument) to initail tensors
        Left_tensors = tf.concat([Left_tensors, tf.zeros(shape = (Left_tensors.shape[0],3), dtype = tf.int32)], axis = 1)
        Right_tensors = tf.concat([Right_tensors, tf.zeros(shape = (Right_tensors.shape[0],3), dtype = tf.int32)], axis = 1)
        #Concat with initial
        Left_final = tf.concat([Left_Positive,Left_tensors],axis = 0)
        Right_final = tf.concat([Right_Positive,Right_tensors],axis = 0)
        #Target length for Left and Right tensor 
        target = int(Negative_class_tensor.shape[0]//2)
        #Select elements to meet target lenght 
        target_length_L = tf.subtract(target, ini_length_L)
        if target_length_L < Left_final.shape[0]:
            current_indices_L = tf.random.shuffle(tf.range(Left_final.shape[0]),seed = 100)
            #tf.print("size of indices L: ",current_indices_L.shape)
            reduced_indices_L = tf.slice(current_indices_L,[1],[target_length_L])
            Left_final = tf.gather(params = Left_final,indices = reduced_indices_L,axis = 0)
        
        target_length_R = tf.subtract(target, ini_length_R)
        if target_length_R < Right_final.shape[0]:
            current_indices_R = tf.random.shuffle(tf.range(Right_final.shape[0]),seed = 80)
            #tf.print("size of indices R: ",target_length_R.shape)
            reduced_indices_R = tf.slice(current_indices_R,[1],[target_length_R])
            Right_final = tf.gather(params = Right_final,indices = reduced_indices_R,axis = 0)
        
        tf.print("------------------------------------------------")
        tf.print("After applying MixUP method, in positive class: ")
        tf.print("   Laterality L view: ",Left_final.shape[0],"images")
        tf.print("   Laterality R view: ",Right_final.shape[0],"images")
        tf.print("After applying MixUP method: positive class: ",Left_final.shape[0]+Right_final.shape[0],"images, negative class: ",Negative_class_tensor.shape[0],"images")
        return tf.concat([Left_final, Right_final, Negative_class_tensor], axis = 0)

    def _get_tensors_for_validation(self,pos_vali_frame,neg_vali_frame):
        pos_class_tensor = tf.convert_to_tensor(pos_vali_frame.values, dtype = tf.int32) 
        neg_class_tensor = tf.convert_to_tensor(neg_vali_frame.values, dtype = tf.int32) 
        pos_class_tensor = tf.concat([pos_class_tensor, tf.zeros(shape = (pos_class_tensor.shape[0],3), dtype = tf.int32) ], axis = 1)
        neg_class_tensor = tf.concat([neg_class_tensor, tf.zeros(shape = (neg_class_tensor.shape[0],3), dtype = tf.int32) ], axis = 1)
        return tf.concat([pos_class_tensor, neg_class_tensor], axis = 0)
    
    def _split_pos_class(self,Total_dataframe):
        #Select True Class
        true_class_L = Total_dataframe.query( 'cancer == 1 and laterality == 0', inplace = False)
        true_class_R = Total_dataframe.query( 'cancer == 1 and laterality == 1', inplace = False)
        #Split directory into train dir and vali dir
        train_true_class_L, vali_true_class_L = train_test_split(true_class_L,
                                               test_size = self.train_vali_ratio,
                                               random_state = self.random_object)
        #Reset index of both sets
        train_true_class_L.reset_index(drop= True, inplace = True)
        vali_true_class_L.reset_index(drop= True, inplace = True)
  
        train_true_class_R, vali_true_class_R = train_test_split(true_class_R,
                                               test_size = self.train_vali_ratio,
                                               random_state = self.random_object)
        #Reset index of both sets
        train_true_class_R.reset_index(drop= True, inplace = True)
        vali_true_class_R.reset_index(drop= True, inplace = True)
        
        #Finalize
        train_true_class = pd.concat([train_true_class_L, train_true_class_R], axis = 0, ignore_index=True)
        vali_true_class = pd.concat([vali_true_class_L, vali_true_class_R], axis = 0, ignore_index=True)
    
        #Return dataframe
        return train_true_class, vali_true_class
    
    def _split_neg_class(self,Total_dataframe):
        #Select True Class
        false_class = Total_dataframe.query( 'cancer == 0', inplace = False)
        #Split directory into train dir and vali dir
        train_false_class, vali_false_class = train_test_split(false_class,
                                               test_size= self.train_vali_ratio,
                                               random_state=self.random_object)
        #Reset index of both sets
        train_false_class.reset_index(drop= True, inplace = True)
        vali_false_class.reset_index(drop= True, inplace = True)
        #Return dataframe
        return train_false_class, vali_false_class

    #@tf.function
    def make_img_dir_list(self):
        #Read csv
        train_list = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
        #Make directory list
        img_dim_list=pd.DataFrame(data={'patient_id': train_list.patient_id
                                    ,'image_id': train_list.image_id
                                    ,'laterality':train_list.laterality
                                    ,'cancer': train_list.cancer
                                    , 'density':train_list.density}
                              ,columns = ['patient_id','image_id','laterality','cancer','density'])
        img_dim_list.density.fillna(0, inplace=True)
        #Convert string value to integer
        #'laterality': 'L'->0, 'R'->1
        #'density': 'A'->1,'B'->2,'C'->3,'D'->4  
        img_dim_list = img_dim_list.replace(['L','R','A','B','C','D'], [0,1,1,2,3,4])
        selected_img_list = img_dim_list.sample(self.total_data_points)
        
        #Split positive and negative class respectively
        pos_train, pos_vali = self._split_pos_class(selected_img_list)
        neg_train, neg_vali = self._split_neg_class(selected_img_list)
        tf.print("Before applying MixUP method: positive class : ",pos_train.shape[0],"images,"," negative class: ",neg_train.shape[0],"images")
        tf.print("Before applying MixUP method: total size of training set : ",pos_train.shape[0]+neg_train.shape[0],"images")
        #Argumentation on train data of 'True' class
        argumented_training_tensor = self._get_mix_tensors_for_training(pos_train,neg_train , duplicate_constant = 2)  
        tf.print("After applying MixUP method: total size of training set: ", argumented_training_tensor.shape[0],"images")
        
        #Add zeors into new columns
        validation_tensor = self._get_tensors_for_validation(pos_vali,neg_vali)
        tf.print("------------------------------------------------")
        tf.print("Total size of validation set: ",validation_tensor.shape[0], "images")
        tf.print("include: positive class: ",pos_vali.shape[0],"images,"," negative class: ",neg_vali.shape[0],"images")
        #Return tensor
        tf.print("End create training and validation tensor")
        return argumented_training_tensor, validation_tensor

In [7]:
class Make_TFRecord(Distribute_train_vali_data):
    def __init__(self,Number_of_cases,target_img_height, target_img_width,train_vali_ratio, dtype, epochs, random_object):
        """
        Class for reading data from original format e.g .dcm file with tensor list containts patient_id and image_id. 
        Then decode in jpeg as bytestring to TFRecord files
        """
        self.random_object = random_object
        super().__init__(Number_of_cases,train_vali_ratio,self.random_object)
        self.main_dtype = dtype
        self.epochs = epochs
        self.target_img_height = target_img_height
        self.target_img_width = target_img_width

        self.zero_scalar = tf.constant(0, dtype = tf.int32, shape = ())
        self.one_scalar = tf.constant(1, dtype = tf.int32, shape = ())
        self._initialize_dataset()
        self._current_TFrecord_path = ""
    
    def _split_value(self, row):
        """Split a row of orginanl data into patiend_id and image_id"""
        return tf.slice(row, [0],[1]), tf.slice(row, [1],[1]) 
    
    def _split_values_of_two_images(self,row):
        return  tf.slice(row, [0],[1]), tf.slice(row, [1],[1]), tf.slice(row, [5],[1]), tf.slice(row, [6],[1]) 
    
    def _dicom_to_tensor(self, patient_id, img_id):
        """
        Read dicom file and return bytes array
        Input: 
        patient_id: tensor type int
        img_id: tensor type int
        Output:
        numpy array of image
        """
        patient_id = tf.reshape(patient_id, shape = ())
        img_id = tf.reshape(img_id, shape = ())
        #tf.print("value: ",patient_id," ,type: ",patient_id.dtype)
        train_default_path = f'/kaggle/input/rsna-breast-cancer-detection/train_images/'
        string_tensor_path = tf.strings.format(train_default_path+'{}/{}.dcm', (patient_id,img_id))
        
        string_np_path = string_tensor_path.numpy().decode()
        dcm_file = dcmread(string_np_path, force = True)
        image_array = dcm_file.pixel_array  #Output from extract dicom is uint16
        image_array = image_array.reshape((image_array.shape[0],image_array.shape[1],1))
        return tf.cast(tf.image.resize(image_array,size = (self.target_img_height,self.target_img_width)), dtype = tf.uint16)
    
    def _get_an_image_bytes(self, pa, img_id):
        """Use for both cases"""
        img_tensor = tf.py_function(func=self._dicom_to_tensor, inp=[pa, img_id], Tout=tf.uint16)
        
        # Normalize to 0 and 1 value
        temp_img = tf.math.divide(img_tensor,tf.constant(32767,dtype = img_tensor.dtype))
        
        # Convert to unit8 range from 0 to 255
        img_tensor = tf.math.multiply(img_tensor, tf.constant(255,dtype = img_tensor.dtype))
        
        # Ready to cast to uint8
        img_tensor = tf.cast(img_tensor, dtype = tf.uint8)
        
        tf.debugging.assert_greater(tf.shape(tf.shape(img_tensor)),2)
        
        buff_tensor = tf.io.encode_jpeg(img_tensor,format='grayscale',quality=95,progressive=False,optimize_size=True, chroma_downsampling=True,density_unit='in',x_density=300, y_density=300,xmp_metadata='')
        return tf.reshape(buff_tensor, shape = (1,))  #Shape = (1,) for creating data.Dataset
    
    def _condition_function_argument(self, a_row):
        return tf.math.equal(tf.slice(a_row,[5],[1]), self.zero_scalar)
    
    def _true_function_argument(self, a_row):
        """
        Use for normal case, no need mixup
        """
        pa,img_id = self._split_value(a_row)
        #Return format: image btyes of main image, image btyes of main image, key =1 for SKIPPING second img bytes
        return self._get_an_image_bytes(pa,img_id), tf.constant([b"\x00"], dtype=tf.string), tf.constant([1],dtype = tf.int32)
    
    def _false_function_argument(self, a_row):
        """
        Use for argumentation case
        """
        pa1,img_id1, pa2, img_id2 = self._split_values_of_two_images(a_row)
        img_bytes_1 = self._get_an_image_bytes(pa1,img_id1)
        img_bytes_2 = self._get_an_image_bytes(pa2,img_id2)
        
        #Return format: image btyes of main image, image btyes of paired image, key =0 for READING second img bytes
        return img_bytes_1 , img_bytes_2, tf.constant([0],dtype = tf.int32)
    
    @tf.function
    def _create_a_tuple(self,an_element):    #map function for Dataset
        tf.debugging.assert_equal(an_element.shape, 8, message="element's shape is not equal 8")
        #tf.print("Input row: ", an_element,", with type: ",an_element.dtype)
        #Condition to check need to do argument or not
        image_bytes_1, image_bytes_2, key= tf.cond(self._condition_function_argument(an_element),
                           lambda: self._true_function_argument(an_element),
                           lambda: self._false_function_argument(an_element))
        
        #Make label for each image
        def assign_label_1():
            #Class 0 for having no cancer
            return tf.constant([0], dtype=tf.int32)
        def assign_label_2():
            #Class 1 for having cancer
            return tf.constant([1], dtype=tf.int32)
        
        #Compare and assign label
        temp_label = tf.cond( tf.equal(tf.slice(an_element,[3],[1]), tf.constant(0)), assign_label_1, assign_label_2 )
        #tf.print("Returned a dataset")
        return tf.data.Dataset.from_tensor_slices((image_bytes_1, temp_label, image_bytes_2, key))
    
    def _create_dataset(self, input_dataset, batch_size):
        return input_dataset.interleave(lambda single_row: self._create_a_tuple(single_row),
                                                       cycle_length = batch_size,
                                                       block_length = batch_size,
                                                       num_parallel_calls=tf.data.AUTOTUNE)
    
    # Three types of data can be stored in TFRecords: bytestrings, integers and floats
    # They are always stored as lists, a single data element will be a list of size 1
    def _bytestring_feature(self,list_of_bytestrings):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))
    
    def _int_feature(self,list_of_ints): # int64
        return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

    def _float_feature(self,list_of_floats): # float32
        return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))

    def _make_example(self, main_img_bytes, label, paired_img_bytes, key):
        feature = {
            "image": self._bytestring_feature([main_img_bytes.numpy()]),          # one image in the list
            "class": self._int_feature([label]),                                  # one class in the list
            "paired_image": self._bytestring_feature([paired_img_bytes.numpy()]), # one image in the list
            "key": self._int_feature([key])                                       # one key in the list
        }
        return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()
    
    @tf.function
    def tf_example(self,main_img_bytes, label, paired_img_bytes, key):
        return tf.py_function(self._make_example,[main_img_bytes, label, paired_img_bytes, key],tf.string)
    
    def _initialize_dataset(self):
        train_dir_tensors, vali_dir_tensors = self.make_img_dir_list()
        self.train_dataset = tf.data.Dataset.from_tensor_slices(train_dir_tensors)
        self.vali_dataset = tf.data.Dataset.from_tensor_slices(vali_dir_tensors)
        self.train_length = train_dir_tensors.shape[0]
        self.vali_length = vali_dir_tensors.shape[0]
        del train_dir_tensors
        del vali_dir_tensors
        gc.collect()        
        
    # Wrapped by tf.py_function, execute in eager mode only
    def _write_a_record(self, contents,batch_order,batch_size_writing_file):
        temp_dataset = tf.data.Dataset.from_tensor_slices(contents)
        tf.print("Size of content: ",tf.shape(contents))
        temp_dataset = temp_dataset.batch(1) #read single row
        #tfrec_file_dir: string tensor
        tfrec_file_dir = tf.strings.join([self._current_TFrecord_path,tf.as_string(batch_order),"_",tf.as_string(batch_size_writing_file),".tfrec"])
        
        Writer_options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=2)
        with tf.io.TFRecordWriter(tfrec_file_dir.numpy().decode(),options = Writer_options) as file_writer:
            #loop through each element in batch for each file
            for single_row in temp_dataset:
                file_writer.write(single_row.numpy()[0])
            file_writer.flush()
        return None
    
    @tf.function(autograph = True)
    def _write_a_record_run_func(self,batch_elements,batch_order,current_batch_size):
        tf.py_function(func = self._write_a_record,inp = [batch_elements,batch_order,current_batch_size], Tout = [])
    
    def _update_TFrecord_path(self, new_TFrecord_path):
        """new_TFrecord_path: python string"""
        self._current_TFrecord_path = new_TFrecord_path
    
    #Normal python function
    def write_to_tfrecord(self,input_dataset, current_batch_size, string_tensor_data_dir, limit_batch_order, strategy):         
        interleaved_dataset = self._create_dataset(input_dataset, current_batch_size)
        #Update save directory of TFrecord file
        self._update_TFrecord_path(string_tensor_data_dir)
        print("Start Writing TFRecords")
        #Map elements in interleaved_dataset to features of the example
        serialized_features_dataset = interleaved_dataset.map(self.tf_example, num_parallel_calls=tf.data.AUTOTUNE)
        
        serialized_features_dataset = serialized_features_dataset.batch(current_batch_size,drop_remainder=True)
        serialized_features_dataset = serialized_features_dataset.prefetch(buffer_size = 3)
        #Distributed the dataset over devices
        dist_dataset = strategy.experimental_distribute_dataset(serialized_features_dataset)
        
        for batch_order, batch_elements in enumerate(dist_dataset):
            if batch_order < limit_batch_order:
                tf.print("Current batch order: ",batch_order)
                strategy.run(self._write_a_record_run_func,args=(batch_elements,
                                                                 tf.constant(batch_order, dtype = tf.int32),
                                                                 tf.constant(current_batch_size, dtype = tf.int32)))
            else:
                break
            batch_elements = None
            del batch_elements
        del serialized_features_dataset, interleaved_dataset
        gc.collect()

In [8]:
# # patient_id = 22895
# # img_id = 1002924244

# patient_id = 10011
# img_id = 1031443799

# train_default_path = f'/kaggle/input/rsna-breast-cancer-detection/train_images/'
# string_tensor_path = tf.strings.format(train_default_path+'{}/{}.dcm', (patient_id,img_id))
        
# string_np_path = string_tensor_path.numpy().decode()
# dcm_file = dcmread(string_np_path, force = True)

# offset = dcm_file.PixelData.find(b"\x00\x00\x00\x0C")
# # ini_pixel = dcm_file.pixel_array
# test_pixel = dcm_file.pixel_array
# tf.print(test_pixel.shape)
# main_img = tf.constant(test_pixel,shape = (test_pixel.shape[0], test_pixel.shape[1],1),dtype = tf.uint8)

# buff_tensor = tf.io.encode_jpeg(main_img,format='grayscale',quality=80,
#                                 progressive=True,optimize_size=True, 
#                                 chroma_downsampling=True,density_unit='in',x_density=300, y_density=300,xmp_metadata='')

# tf.print("String type: ",tf.io.is_jpeg(buff_tensor))

# decoded_image = tf.io.decode_jpeg(buff_tensor,channels=1,ratio=1,
#                                          fancy_upscaling=True,
#                                          try_recover_truncated=True,
#                                          acceptable_fraction=1,dct_method='')
# decoded_image = tf.cast(decoded_image, dtype = tf.uint8)

# tf.print(decoded_image.numpy().shape)
# tf.print(np.array_equal(main_img, decoded_image.numpy()))
# plt.imshow(decoded_image.numpy(),cmap ="gray")
# plt.show()


# feature = {"image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[buff_tensor.numpy()] )) }
# an_example = tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

# data = tf.data.Dataset.from_tensor_slices([an_example, an_example])

# data = data.batch(1) #read single row
        
# tfrec_file_dir = tf.strings.join(["/kaggle/working/1_dump.tfrec"])
        
# # Writer_options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=9) #options = Writer_options
# # with tf.io.TFRecordWriter(tfrec_file_dir.numpy().decode()) as file_writer:
# #     #loop through each element in batch for each file
# #     for single_row in data:
# #         file_writer.write(single_row.numpy()[0])
# #         file_writer.flush()

In [9]:
class Main_Architecture(tf.keras.Model, tf.keras.callbacks.Callback):
    
    def __init__(self,img_input_shape, optimizer_params, dtype):
        super().__init__()
#
        self.img_input_shape = img_input_shape
        self.optimizer_params = optimizer_params
        self.data_type = dtype
        self.input_layer = tf.keras.Input(self.img_input_shape, name='image', dtype= self.data_type)
        self.random_flip = tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=250)
        
#         self.body = swin_transformer_v2.SwinTransformerV2Tiny_window16(input_shape=self.img_input_shape,
#                                                      num_classes=0, 
#                                                      classifier_activation = None, 
#                                                      pretrained=None)
        #Conv2D initialization by variance scaling method
        self.body = tf.keras.applications.efficientnet_v2.EfficientNetV2B1(
                                include_top=None,
                                weights=None,
                                #input_tensor=self.input_layer,
                                input_shape=self.img_input_shape,
                                pooling=None,
                                classes=0,
                                classifier_activation=None,
                                include_preprocessing=None)
        
        self.Dense = tf.keras.layers.Dense(units = 512, 
                                           activation='relu',
                                           kernel_initializer='glorot_uniform')   
        
        self.pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(0.30)
        self.dense_out = tf.keras.layers.Dense(units = 2, activation='softmax',kernel_initializer='glorot_uniform') 
        
    def call(self, training, return_covmat):
        #Add arugmentation layer
        argu_layer = self.random_flip(self.input_layer)
        # Add Swin transformer
        body =  self.body(argu_layer)
        # Average Pooling 
        conv_out = self.pooling(body)  
        
        output= self.dropout(conv_out)
        output = self.dense_out(output)
        return output
    
    def create_model(self):
        
        output = self.call(training = False,  return_covmat=False)
        my_model = tf.keras.Model(inputs = self.input_layer, outputs = output)
        # Optimizer    
        optimize = tf.keras.optimizers.Adam(**self.optimizer_params)
        # Loss
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        # Compile model
        my_model.compile(optimizer = optimize,
                         loss = loss_fn,
                         metrics = ['accuracy','AUC']
                        )
        
        my_model.summary()
        return my_model

In [10]:
class PROCESS_TENSOR:
    def __init__(self):
        #Initilize value
        self.zero_scalar = tf.constant(0, dtype = tf.int32, shape = ())
        tf.print("Process tensor class is init!")
   
    def _normalize_tensor(self,input_tensor, max_val, minval):
        """all dtype: tensor"""
        return tf.math.divide(tf.math.subtract(input_tensor, minval),tf.math.subtract(max_val,minval))
    
    def Gen_x1_x2(self, lower_segment, upper_segment, current_img_width):
        #Find length
        lower_seg_length = tf.math.count_nonzero(lower_segment)
        upper_seg_length = tf.math.count_nonzero(upper_segment)
        
        def fnc1():
            #tf.print("True func")
            return  lower_segment
        def fnc2():
            #tf.print("False func")
            return  upper_segment
        out_seg = tf.cond(tf.math.greater(lower_seg_length, upper_seg_length), fnc1,fnc2 )
        
        #Select input segment for compute
        max_id = tf.math.argmax(out_seg,output_type = tf.int32)
        min_id = tf.math.argmin(out_seg,output_type = tf.int32)
        max_id = tf.reshape(max_id, shape=())
        min_id = tf.reshape(min_id, shape=())
        
#         condition1 = tf.math.greater_equal( current_img_width, max_id)
#         tf.debugging.Assert(condition1, [current_img_width, max_id, tf.shape(out_seg), image_id])
        
        def correct_x1():
            #Remove end, select head
            return self.zero_scalar , max_id
        def correct_x2():
            #Remove head, select end
            return min_id , tf.math.subtract(tf.cast(current_img_width,dtype = tf.int32), min_id)
        
        cond_argmax_min =  tf.math.logical_and( tf.math.less( min_id , max_id ), tf.math.greater( max_id, self.zero_scalar) )
        return tf.cond(cond_argmax_min, correct_x1, correct_x2) 
        
    def Get_height_offset_target(self,height_dim,  input_dtype=tf.float32, output_dtype = tf.int32):
        height_dim = tf.cast(height_dim, dtype =  input_dtype)
        offset_height_ratio = tf.constant(0.15, dtype = input_dtype)
        offset_height = tf.math.multiply(height_dim, offset_height_ratio)
        target_height = tf.math.subtract(height_dim ,offset_height)
        
        return tf.cast(offset_height, dtype = output_dtype), tf.cast(target_height, dtype = output_dtype)
        
    #Crop image in tensor version
    @tf.function
    def Crop_Resize_tensor(self,img_tensor_input_value, bound , target_size):
        """
        Operate crop img with tensor, not numpy
        Input: 
        img_tensor_input: tf.Variable
        curr_img_H: tensor
        curr_img_W: tensor
        bound: upper bound and lower bound
        Output:
        crped_img: tensor
        """
        #Get current dimension
        #tf.print(tf.shape(img_tensor_input_value))
        curr_img_H = tf.slice(tf.shape(img_tensor_input_value),[0],[1])
        curr_img_W = tf.slice(tf.shape(img_tensor_input_value),[1],[1])
        curr_img_H = tf.reshape(curr_img_H, shape = ())
        curr_img_W = tf.reshape(curr_img_W, shape = ())
        #Processing image level
        #Normailize img
        #input: dicom image with maximum value is 32767, output: image tensor range from 0 to 1
        img_dtype = img_tensor_input_value.dtype
        img_tensor_input_value_norm =  self._normalize_tensor(img_tensor_input_value,
                                                              tf.constant(32767, dtype = img_dtype),
                                                              tf.constant(0, dtype = img_dtype)) 
        
        # Processing histogram
        # Calculate histogram (sum of pixel value along axis 0) of image
        hist = tf.math.reduce_sum(img_tensor_input_value_norm, axis=0, keepdims=True)  # Have to keepdims to retain original shape
        # Normailize hist
        # input: histogram tensor dtype tf.float16 or the same type as input image tensor
        norm_hist = self._normalize_tensor(hist, tf.math.reduce_max(hist),tf.constant(0, dtype = hist.dtype))
        norm_hist = tf.squeeze(input = norm_hist)
        # Find each segment
        lower_seg = tf.where(tf.subtract(norm_hist,bound[0]) < 0,tf.constant(1),tf.constant(0))
        upper_seg = tf.where(tf.subtract(norm_hist,bound[1]) > 0,tf.constant(1),tf.constant(0))
        # Gen x1 and x2
#         condition = tf.math.logical_and( tf.math.equal(shape1 ,curr_img_W) , tf.math.equal(shape2,curr_img_W) )
#         tf.debugging.Assert(condition, [tf.shape(lower_seg), tf.shape(upper_seg), curr_img_W, tf.shape(hist)])
        offset, target = self.Gen_x1_x2(lower_seg, upper_seg, curr_img_W)
        offset_H, target_H = self.Get_height_offset_target(curr_img_H)
        #Cropping
        img_tensor_input_value2 =  tf.image.crop_to_bounding_box(
                                                img_tensor_input_value_norm,
                                                offset_height = offset_H,
                                                offset_width = offset,
                                                target_height = target_H,
                                                target_width = target
                                                             )
 
        # Resize output
        # Method: tensorflow.org/api_docs/python/tf/image/ResizeMethod
        return tf.image.resize(
                                img_tensor_input_value2,
                                size = target_size,
                                method=tf.image.ResizeMethod.BILINEAR,
                                preserve_aspect_ratio=False,
                                antialias=True
                             )

In [11]:
class LOADER(PROCESS_TENSOR):
    def __init__(self, target_height, target_width, master_batch_size,writing_TRF_batch_size, bound, dtype, tfrecord_img_dir, epochs):
        """
        img_dir_tensor: tensor
        """
        self.target_height = target_height
        self.target_width = target_width
        self.master_batch_size = master_batch_size
        self.bound = bound
        self.main_dtype = dtype
        self.tfrecord_img_dir = tfrecord_img_dir
        self.writing_TRF_batch_size = writing_TRF_batch_size
        self.epochs = epochs
        PROCESS_TENSOR.__init__(self)
    
    def _decode_and_cast(self, buff_string):
        decoded_image = tf.io.decode_jpeg(buff_string,channels=1,ratio=1, fancy_upscaling=True,try_recover_truncated=True,acceptable_fraction=1,dct_method='')
        return tf.cast(decoded_image, dtype = self.main_dtype)
        
    def _sample_beta_distribution(self,concentration_0=0.2, concentration_1=0.2, seed_value = 100):
        gamma_1_sample = tf.random.gamma(shape=[1], alpha=concentration_1, seed = seed_value)
        gamma_2_sample = tf.random.gamma(shape=[1], alpha=concentration_0, seed = seed_value)
        return tf.cast(tf.math.divide(gamma_1_sample ,tf.math.add(gamma_1_sample ,gamma_2_sample)) , dtype = self.main_dtype)
    
    def _mix_up_argument(self, img1_tensor, img2_tensor):
        """
        img1 and img2 are two images that has already pre-processing and has same dimension
        """
        lambda_value = self._sample_beta_distribution(concentration_0 = 0.4)
        #lambda_value = tf.constant(1,dtype = self.main_dtype)
        minus_lambda_value = tf.subtract(tf.constant(1, dtype = self.main_dtype),lambda_value)
        return tf.math.add(tf.math.multiply(img1_tensor,lambda_value), tf.math.multiply(img2_tensor,minus_lambda_value))
      
        
    def condition_function_argument(self, an_example):
        return tf.math.equal(an_example["key"], tf.constant([1],dtype = tf.int64))
    
    def true_function_argument(self, an_example):
        """
        Use for normal case, no need mixup
        """
        #Read and decode main image
        #tf.print("True case")
        main_image = self._decode_and_cast(an_example["image"])
        #main_image = tf.reshape(main_image, shape = (1,main_image.shape[0],main_image.shape[1],1))
        return self.Crop_Resize_tensor(img_tensor_input_value= main_image,
                                                           bound = self.bound,
                                                           target_size = [self.target_height,self.target_width]) 
    
    def false_function_argument(self, an_example):
        """
        Use for need of argumentation case
        """
        #Read and decode main image
        #tf.print("False case")
        main_image = self._decode_and_cast(an_example["image"])
        #main_image = tf.reshape(main_image, shape = (1,main_image.shape[0],main_image.shape[1],1))
        main_image = self.Crop_Resize_tensor(img_tensor_input_value= main_image,
                                                           bound = self.bound,
                                                           target_size = [self.target_height,self.target_width]) 
        
        #Read and decode paired image
        paired_image = self._decode_and_cast(an_example["paired_image"])
        #paired_image = tf.reshape(paired_image, shape = (1,paired_image.shape[0],paired_image.shape[1],1))
        paired_image = self.Crop_Resize_tensor(img_tensor_input_value= paired_image,
                                                           bound = self.bound,
                                                           target_size = [self.target_height,self.target_width]) 
        
        return self._mix_up_argument(main_image, paired_image)
        
    @tf.function
    def _read_tfrecord(self,example_row):
        tfrecord_format = {
                "image": tf.io.FixedLenFeature([], tf.string),
                "class": tf.io.FixedLenFeature([], tf.int64),
                "paired_image": tf.io.FixedLenFeature([], tf.string),
                "key": tf.io.FixedLenFeature([], tf.int64)
            }
        label_format = tf.constant([[1,0],[0,1]], dtype = tf.int32)
        # Get an example
        example = tf.io.parse_single_example(example_row, tfrecord_format)
        
        #Read and decode label, class
        label = tf.gather(params = label_format,indices = example["class"])
        
        #Read, decode and mix images
        temp_img = tf.cond(self.condition_function_argument(example),
                           lambda: self.true_function_argument(example),
                           lambda: self.false_function_argument(example))
        tf.debugging.assert_greater_equal(temp_img, tf.constant(0, temp_img.dtype), message=None, summarize=None)
        return (temp_img, label)
    
    def simple_plot(self, img_index):
        files_path = tf.io.gfile.glob("/kaggle/input/dataset-ver1/training/11_900.tfrec")
        dataset = tf.data.TFRecordDataset(files_path,
                                          compression_type= 'GZIP',
                                          buffer_size = 300,
                                          num_parallel_reads = 4)  
        dataset = dataset.map(lambda x : self._read_tfrecord(x),
                              num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.take(3)
        dataset = dataset.enumerate(start = 0)
        #tf.print("Lenght of dataset: ",)
        for order, (image,label) in dataset.as_numpy_iterator():
            if order == img_index :
                tf.print("image shape: ",image.shape)
                tf.print("image type: ",image.dtype)
                tf.print("label: ", label)
                
                image = tf.math.multiply(image,tf.constant(32767,dtype = image.dtype))
                #rescle data for plotting
                image = tf.cast(image, dtype = tf.float16)
    
                plt.imshow(image, cmap = "gray")
                plt.show
    
    def create_dataset(self,strategy, num_replicas):
        options = tf.data.Options()
        options.threading.private_threadpool_size = 3
        options.experimental_deterministic = False  # disable order, increase speed
        files_path = tf.io.gfile.glob(self.tfrecord_img_dir + "*.tfrec")
        tf.print("File list: ", files_path)
        dataset = tf.data.TFRecordDataset(files_path,
                                          compression_type= 'GZIP',
                                          buffer_size = 120,
                                          num_parallel_reads = 4)  # automatically interleaves reads from multiple files
        
        dataset = dataset.with_options(options)  # uses data as soon as it streams in, rather than in its original order
        dataset = dataset.map(lambda x : self._read_tfrecord(x),
                              num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.batch(self.master_batch_size,drop_remainder = True)
        
        dataset = dataset.prefetch(buffer_size = tf.data.AUTOTUNE)
        dataset = dataset.repeat(self.epochs)
        #Find actual length of input dataset
        dataset_length = int(len(files_path)*self.writing_TRF_batch_size//(self.master_batch_size*num_replicas))
            #Distributed the dataset over devices
        #dist_dataset = strategy.experimental_distribute_dataset(dataset)
        #dataset = dataset.shuffle(buffer_size = dataset_length)
        
        return dist_dataset , dataset_length

In [12]:
def scheduler(epoch, lr):
    if epoch > 2:
        return lr
    else:
        return lr * tf.math.exp(-0.2)

def Training_model(model_version, model_inst, train_gen_obj, vali_gen_obj, EPOCHS,train_steps, vali_steps, VERBOSE, num_of_workers):
    
    model = model_inst.create_model()
    
    #Learning rate callback
    LR_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    
    # Define the ProgbarLogger callback
    progbar_callback = tf.keras.callbacks.ProgbarLogger(count_mode= 'samples',stateful_metrics='accuracy')
    # Checkpoint callback
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                        filepath='/kaggle/working/best_weight_'+model_version+'.h5',
                                        save_weights_only=True,
                                        monitor='val_accuracy',
                                        mode='max',
                                        verbose = 1,
                                        save_best_only=True)
 
    # Fit model with data
    history = model.fit(
            x=train_gen_obj,
            epochs=EPOCHS,
            verbose=1,
            callbacks=[model_checkpoint_callback, progbar_callback, LR_callback],
            validation_data=vali_gen_obj,
            workers = num_of_workers,
            use_multiprocessing = False,
            steps_per_epoch= train_steps,
            validation_steps = vali_steps
            )
    return history

In [13]:
# Parameter
k = -2
img_height = 1024 - k*128
img_width = 512 - k*64
img_shape = (img_height, img_width,1)
epochs = 300
verbose = 1

#gcs_path = KaggleDatasets().get_gcs_path("rsna-breast-cancer-detection")
#train_default_path = f'{gcs_path}/train_images/'
train_default_path = f'/kaggle/input/rsna-breast-cancer-detection/train_images/'
test_default_path = f'/kaggle/input/rsna-breast-cancer-detection/test_images/'
# # Test file paths
#DICOM_FILE_PATHS = sorted(tf.io.gfile.glob(f'{gcs_path}/train_images/*/*.dcm'))
#print(f'Found {len(DICOM_FILE_PATHS)} DICOM files')

tfrecord_train_path = f'/kaggle/input/dataset-ver1/training/'
tfrecord_vali_path = f'/kaggle/input/dataset-ver1/validation/'
#num_replicas_in_sync = strategy.num_replicas_in_sync
#Batch file for writing file faster
train_TFR_batch_size = 600
vali_TFR_batch_size = 200

#Batch for training process
train_vali_ratio = 0.4
num_replicas_in_sync = strategy.num_replicas_in_sync
vali_batch_size = 8*num_replicas_in_sync   
train_batch_size = int(vali_batch_size//train_vali_ratio)

ver_model = "ver_2"

seed = 150
bound = [0.2, 0.8]
optimizer_params = dict(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7, weight_decay = 0.00001*0.01)
num_of_workers = 2
master_img_dtype = tf.float32
#Create random object for shullfe
rand_obj = np.random.RandomState(seed = seed)

#Check directory
if os.path.exists('/kaggle/working/training') == False:
    os.makedirs('/kaggle/working/training')
if os.path.exists('/kaggle/working/validation') == False:
    os.makedirs('/kaggle/working/validation')

In [ ]:
with strategy.scope(): 
    #Runing session
    main_instance = Make_TFRecord(20000,img_height, img_width,train_vali_ratio, master_img_dtype, epochs, rand_obj)
    main_instance.write_to_tfrecord(main_instance.vali_dataset,
                                    vali_TFR_batch_size,
                                    '/kaggle/working/validation',
                                    50,
                                    strategy)
    
    main_instance.write_to_tfrecord(main_instance.train_dataset,
                                    train_TFR_batch_size,
                                    '/kaggle/working/training', 
                                    100,
                                    strategy)
    tf.print("Finish writing TFRecords")
    
    main_instance = None
    del main_instance

Before applying MixUP method: positive class :  251 images,  negative class:  11748 images
Before applying MixUP method: total size of training set :  11999 images
Befor applying MixUP method, in positive class: 
   Laterality L view:  125 images
   Laterality R view:  126 images


In [ ]:
#Master training model
with strategy.scope():
    model_instance = Main_Architecture(img_input_shape = img_shape, optimizer_params = optimizer_params, dtype = master_img_dtype)
    
    # Create data loading object   target_height, target_width, master_batch_size, bound, dtype, tfrecord_img_dir
    Train_loader = LOADER(img_height,img_width, train_batch_size, train_TFR_batch_size, bound, master_img_dtype, tfrecord_train_path , epochs)
    Vali_loader = LOADER(img_height,img_width, vali_batch_size, vali_TFR_batch_size, bound, master_img_dtype, tfrecord_vali_path, epochs)
    # Return a dataset
    train_dataset, train_steps = Train_loader.create_dataset(strategy, num_replicas_in_sync)
    vali_dataset, vali_steps = Vali_loader.create_dataset(strategy, num_replicas_in_sync)
    
    tf.print("Ready for training")
    #Training model
    history_train = Training_model(ver_model,model_instance, train_dataset, vali_dataset, 
                                       epochs,
                                       train_steps,
                                       vali_steps,
                                       verbose, num_of_workers)
    #summarize history for accuracy
    plt.plot(history_train.history['accuracy'])
    plt.plot(history_train.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    plt.show()
    # summarize history for loss
    plt.plot(history_train.history['loss'])
    plt.plot(history_train.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    train_dataset = None
    vali_dataset = None
    
    del Train_loader
    del Vali_loader

In [ ]:
def _extract_infor():
    test_list = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
    test_frame = pd.DataFrame(data={'patient_id': test_list.patient_id
                                    ,'image_id': test_list.image_id}
                              ,columns = ['patient_id','image_id'])
    return tf.convert_to_tensor(test_frame.values, dtype = tf.int32)  
 
def _dicom_to_image_test(infor_tensor,target_img_height,target_img_width):
    patient_id = tf.slice(infor_tensor,[0],[1])
    img_id = tf.slice(infor_tensor,[1],[1])
    patient_id = tf.reshape(patient_id, shape = ())
    img_id = tf.reshape(img_id, shape = ())
    train_default_path = f'/kaggle/input/rsna-breast-cancer-detection/test_images/'
    string_tensor_path = tf.strings.format(train_default_path+'{}/{}.dcm', (patient_id,img_id))
    string_np_path = string_tensor_path.numpy().decode()
    dcm_file = dcmread(string_np_path, force = True)
    image_array = dcm_file.pixel_array
    image_array = image_array.reshape((image_array.shape[0],image_array.shape[1],1))
    return tf.cast(tf.image.resize(image_array,size = (target_img_height,target_img_width)), dtype = tf.float16)   

def _dicom_to_image(infor_tensor,target_img_height,target_img_width):
    return tf.py_function(_dicom_to_image_test,inp=[infor_tensor,target_img_height,target_img_width],Tout = tf.float16)
    
    
def create_dataset_and_processing(target_img_height,target_img_width, bound):
    test_dataset = tf.data.Dataset.from_tensor_slices(_extract_infor())
    test_dataset = test_dataset.map(lambda x: _dicom_to_image(x,target_img_height,target_img_width))
    process_tensor_instance = PROCESS_TENSOR()
    test_dataset = test_dataset.map(lambda y: process_tensor_instance.Crop_Resize_tensor(y,bound , [target_img_height,target_img_width]))
    
    return test_dataset

def Inference():
    #Init new instance
    model_instance = Main_Architecture(img_input_shape = img_shape, optimizer_params = optimizer_params, dtype = master_img_dtype)
    inference_model = model_instance.create_model()
    #Load trained weights
    inference_model.load_weights('/kaggle/working/best_weight_ver_2.h5')
    
    #Make predict on processed images in form of tf.data.Dataset
    return inference_model.predict(create_dataset_and_processing())

In [ ]:
import pandas as pd
import tensorflow as tf
# sample_submission = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv')

# display(sample_submission.info())
# display(sample_submission.head())

scores = Inference()
scores = tf.slice(scores, [0,1],[scores.shape[0],1])
scores = pd.DataFrame(scores, columns = ['cancer']).astype("float")
tf.print(scores)
test_list = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
test_result = pd.DataFrame(data={'patient_id': test_list.patient_id
                                    ,'image_id': test_list.image_id
                                    ,'prediction_id': test_list.prediction_id
                                    , 'cancer': scores.cancer}
                              ,columns = ['patient_id','image_id','prediction_id','cancer'])
# Save submission as CSV
test_result.to_csv('submission.csv', index=False)
# show result
pd.read_csv('/kaggle/working/submission.csv')